In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import pandas as pd
from keras.utils import to_categorical
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from Preprocessing import remove_punctuation,remove_stopword,remove_repeated_words
from keras.utils import plot_model
from keras.layers import Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from pyvi import ViTokenizer
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def readData(path):
    df = pd.read_csv(path, encoding='utf-8')
    return df["comment"], df["label"]

In [4]:
X_train, y_train = readData("G:\\VietNamese-Sentiment-Analyst\\data\\data_original\\Train.csv")
X_test, y_test = readData("G:\\VietNamese-Sentiment-Analyst\\data\\data_original\\Test.csv")

In [5]:
print(f"X_Train size:{X_train.shape}")
print(f"y_train size {y_train.shape}")

X_Train size:(7786,)
y_train size (7786,)


In [6]:
def wordseparation(comment):
    return [review.split() for review in comment]

In [7]:
X_train = wordseparation(X_train)
X_test = wordseparation(X_test)
print(f"X_Train: {X_train[0:2]}")
print(f"X_test: {X_test[0:2]}")

X_Train: [['Mới', 'mua', 'máy', 'này', 'Tại', 'thegioididong', 'thốt', 'nốt', 'cảm', 'thấy', 'ok', 'bin', 'trâu', 'chụp', 'ảnh', 'đẹp', 'loa', 'nghe', 'to', 'bắt', 'wf', 'khỏe', 'sóng', 'ổn', 'định,', 'giá', 'thành', 'vừa', 'với', 'túi', 'tiền,', 'nhân', 'viên', 'tư', 'vấn', 'nhiệt', 'tình'], ['Pin', 'kém', 'còn', 'lại', 'miễn', 'chê', 'mua', '8/3/2019', 'tình', 'trạng', 'pin', 'còn', '88%', 'có', 'ai', 'giống', 'tôi', 'không']]
X_test: [['Điện', 'thoải', 'ổn.', 'Facelock', 'cực', 'nhanh,', 'vân', 'tay', 'ôk', ',', 'màn', 'hình', 'lớn,', 'pin', 'trâu', '(', 'liên', 'quân', ',', 'Zalo,', 'YouTube', ')', 'một', 'ngày', 'mất', 'khoảng', '45', '%', ')', 'tuy', 'chỉ', 'chip', '439', 'nhưng', 'rất', 'mượt.', 'Đa', 'nhiệm', 'khá', 'ổn'], ['Mình', 'mới', 'mua', 'vivo91c.', 'Tải', 'ứng', 'dụng', ',games', 'nhanh.', 'Có', 'cái', 'không', 'hài', 'lòng', 'là', 'cài', 'hình', 'nền', 'khóa', 'màn', 'hình', 'không', 'được.', 'Hay', 'tại', 'mình', 'chưa', 'biết', 'hết', 'chức', 'năng', 'của', 'nó.', '

In [8]:
# Create Corpus
def createCorpus(Vi):
    tokenizer = Tokenizer(oov_token="<oov>")
    tokenizer.fit_on_texts(Vi)
    return tokenizer

In [9]:
X_train_corpus = createCorpus(X_train)
X_test_corpus = createCorpus(X_test)

In [10]:
print(X_train_corpus.word_index)
print(X_test_corpus.word_index)

{'<oov>': 1, 'máy': 2, 'mua': 3, 'pin': 4, 'mình': 5, 'game': 6, 'dùng': 7, 'mới': 8, 'tốt': 9, 'chơi': 10, 'ko': 11, 'quá': 12, 'ok': 13, '1': 14, 'đẹp': 15, 'giá': 16, 'nhanh': 17, 'sạc': 18, 'mượt': 19, 'ngày': 20, 'ổn': 21, 'còn': 22, 'camera': 23, 'chụp': 24, 'thấy': 25, 'tầm': 26, 'xài': 27, 'k': 28, 'hơn': 29, 'về': 30, 'màn_hình': 31, 'trâu': 32, 'trong': 33, 'hơi': 34, 'đc': 35, 'khá': 36, 'sản_phẩm': 37, '2': 38, 'lỗi': 39, 'tay': 40, 'tháng': 41, 'nhân_viên': 42, 'hay': 43, 'mọi': 44, 'sử_dụng': 45, 'con': 46, 'sao': 47, 'sài': 48, 'hình': 49, 'hết': 50, 'vân': 51, 'nói_chung': 52, 'nhiệt_tình': 53, 'ngon': 54, 'thứ': 55, 'nóng': 56, 'lắm': 57, '3': 58, 'nghe': 59, 'nó': 60, 'luôn': 61, 'loa': 62, 'điện_thoại': 63, 'tệ': 64, 'khác': 65, 'ảnh': 66, 'người': 67, 'lag': 68, 'wifi': 69, 'mấy': 70, 'ở': 71, 'biết': 72, 'bạn': 73, 'đổi': 74, 'xem': 75, 'dc': 76, 'cấu_hình': 77, 'lần': 78, 'nào': 79, 'hàng': 80, 'sáng': 81, 'tôi': 82, 'đt': 83, 'nói': 84, 'cả': 85, 'tuần': 86, 'đến

In [30]:
# saving
with open('G:\\VietNamese-Sentiment-Analyst\\word_index\\tokenizer_train.pickle', 'wb') as handle:
  pickle.dump(X_train_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
with open('G:\\VietNamese-Sentiment-Analyst\\word_index\\tokenizer_test.pickle', 'wb') as handle:
    pickle.dump(X_test_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
from collections import defaultdict
import re

def loadPickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
        return data

In [13]:
def find_abbreviations(word_dict):
    abbreviations = {}
    for word in word_dict.keys():
        # Kiểm tra các từ viết tắt phổ biến
        if re.match(r'^([A-Z][A-Z]+)$', word) or re.match(r'^[a-z]{1,2}$', word):
            abbreviations[word] = word_dict[word]
    return abbreviations


In [14]:
tokenizer_train = loadPickle('G:\\VietNamese-Sentiment-Analyst\\word_index\\tokenizer_train.pickle') 
tokenizer_test = loadPickle('G:\\VietNamese-Sentiment-Analyst\\word_index\\tokenizer_test.pickle') 

In [15]:
word_dict_test = tokenizer_test.word_index  # Truy cập từ điển từ trong đối tượng Tokenizer
word_dict_train = tokenizer_train.word_index

In [16]:
abbreviation_dict_test = find_abbreviations(word_dict_test)
abbreviation_dict_train = find_abbreviations(word_dict_train)

In [17]:
print("Từ điển từ tập Train:")
print(list(word_dict_train.keys())[10:20])  
print("Từ điển từ tập test:")
print(list(word_dict_test.keys())[10:20]) 

Từ điển từ tập Train:
['ko', 'quá', 'ok', '1', 'đẹp', 'giá', 'nhanh', 'sạc', 'mượt', 'ngày']
Từ điển từ tập test:
['chơi', 'quá', 'ok', '1', 'giá', 'đẹp', 'sạc', 'nhanh', 'mượt', 'chụp']


In [18]:
# Kiểm tra các từ viết tắt và gom vào từ điển abbreviation_dict
abbreviation_dict_Train = defaultdict(int)
abbreviation_dict_Test = defaultdict(int)

for abbr, count in abbreviation_dict_test.items():
    abbreviation_dict_Train[abbr] += count

for abbr, count in abbreviation_dict_train.items():
    abbreviation_dict_Test[abbr] += count

# Lưu từ viết tắt vào tập tin văn bản
with open('G:\\VietNamese-Sentiment-Analyst\\word_index\\abbreviations_train.txt', 'w', encoding='utf-8') as f:
    for abbr in abbreviation_dict_Train.keys():
        f.write(f"{abbr}\n")
        
with open('G:\\VietNamese-Sentiment-Analyst\\word_index\\abbreviations_test.txt', 'w', encoding='utf-8') as f:
    for abbr in abbreviation_dict_Test.keys():
        f.write(f"{abbr}\n")

print("Danh sách từ viết tắt đã được lưu.")

Danh sách từ viết tắt đã được lưu.


In [8]:
def viet_tat_to_full(text):
    # Từ điển chứa các từ viết tắt và dạng đầy đủ
    viet_tat_dict = {
        "chs": "chơi",
        "ko": "không",
        "ok": "ổn",
        "k": "không",
        "dc": "được",
        "e": "em",
        "vs": "với",
        "fb": "Facebook",
        "sp": "sản phẩm",
        "dt": "điện thoại",
        "j": "gì",
        "r": "rồi",
        "ss": "so sánh",
        "sd": "sử dụng",
        "nv": "nhân viên",
        "ae": "anh em",
        "mn": "mọi người",
        "hd": "hoạt động",
        "kg": "kilogram",
        "mk": "mình",
        "v": "vậy",
        "h": "giờ",
        "bt": "bình thường",
        "m": "mình",
        "a": "anh",
        "cx": "cũng",
        "kh": "không",
        "lq": "liên quan",
        "ng": "người",
        "ip": "Iphone",
        "bh": "bây giờ",
        "hk": "hong",
        "tr": "triệu",
        "ad": "admin",
        "wa": "qua",
        "ms": "mới",
        "dk": "điều kiện",
        "t": "tôi",
        "vn": "Việt Nam",
        "d": "đi",
        "b": "bạn",
        "ns": "nói",
        "tp": "thành phố",
        "tn": "tin nhắn",
        "s": "sao",
        "se": "sẽ",
        "gg": "Google",
        "of": "của",
        "up": "lên",
        "z": "vậy",
        "la": "là",
        "nt": "nhắn tin",
        "qc": "quảng cáo",
        "on": "online",
        "sx": "sản xuất",
        "vv": "vân vân",
        "y": "ý",
        "c": "cái",
        "lm": "làm",
        "ah": "à",
        "no": "không",
        "nc": "nói chuyện",
        "gb": "goodbye",
        "n": "nhé",
        "cv": "công việc",
        "wf": "wifi",
        "gi": "gì",
        "va": "và",
        "de": "để",
        "zl": "zalo",
        "kb": "không biết",
        "ik": "đi",
        "ji": "gì",
        "co": "có",
        "ak": "à",
        "tg": "thời gian",
        "bi": "bị",
        "bn": "bạn",
        "vi": "vì",
        "o": "ở",
        "tt": "tình trạng",
        "um": "ừm",
        "yt": "YouTube",
        "ps": "postscript",
        "id": "ID",
        "p": "phần",
        "g": "gram",
        "tv": "television",
        "xl": "xin lỗi",
        "cu": "cứ",
        "da": "đã",
        "tl": "trả lời",
        "x": "xem",
        "xg": "xong",
        "xi": "xì",
        "l": "lít",
        "in": "inch",
        "ve": "về",
        "th": "thôi",
        "cg": "cũng",
        "it": "ít",
        "pk": "phải không",
        "bo": "bỏ",
        "bk": "biết",
        "ch": "chưa",
        "ht": "hết",
        "xr": "xong rồi",
        "kq": "kết quả",
        "ib": "nhắn tin",
        "vd": "ví dụ",
        "ao": "áo",
        "ap": "áp",
        "hn": "hôm nay",
        "su": "sự",
        "re": "rẻ",
        "dp": "đẹp",
        "gt": "giới thiệu",
        "vc": "việc",
        "bb": "bạn bè",
        "cn": "Chủ nhật",
        "du": "đủ",
        "tu": "từ",
        "hi": "xin chào",
        "pt": "phát triển",
        "km": "kilomet",
        "by": "bye",
        "or": "hoặc",
        "mb": "mấy bạn",
        "ke": "kể",
        "lp": "lập",
        "bu": "bù",
        "tq": "trung quốc",
        "lg": "lớn",
        "oy": "gòi",
        "vu": "vụ",
        "bl": "bình luận",
        "xo": "xô",
        "ro": "rõ",
        "hs": "học sinh",
        "uu": "ưu",
        "ru": "rủ",
        "lx": "lâu",
        "ag": "An Giang",
        "vo": "vô",
        "sm": "small",
        "vl": "vãi",
        "ol": "online",
        "ky": "ký",
        "pc": "personal computer",
        "oi": "ơi",
        "jo": "giờ",
        "mp": "mp3",
        "nh": "nhà",
        "uk": "ử",
        "pr": "public relations",
        "vk": "vợ",
        "tc": "tình cảm",
        "qr": "QR code",
        "kt": "kết thúc",
        "tb": "thông báo",
        "sf": "so far",
        "gh": "giao hàng",
        "hg": "không",
        "cb": "chuẩn bị",
        "nx": "nữa",
        "qq": "quần què",
        "bd": "bắt đầu",
        "ni": "này"
    }
    
    # Tách văn bản thành các từ riêng lẻ
    words = text.split()
    
    # Thay thế từ viết tắt bằng dạng đầy đủ
    for i in range(len(words)):
        if words[i] in viet_tat_dict:
            words[i] = viet_tat_dict[words[i]]
    
    # Ghép các từ lại thành câu
    return ' '.join(words)


# Áp dụng hàm cho cột 'input' trong DataFrame
X_train = X_train.apply(viet_tat_to_full)

# Hiển thị DataFrame sau khi chuyển đổi
print(X_train)

AttributeError: 'list' object has no attribute 'apply'

In [33]:
def viet_tat_to_full(text):
    # Từ điển chứa các từ viết tắt và dạng đầy đủ
    viet_tat_dict = {
    "đt": "điện thoại",
    "ko": "không",
    "ok": "ổn",
    "k": "không",
    "dc": "được",
    "e": "em",
    "vs": "với",
    "sp": "sản phẩm",
    "fb": "Facebook",
    "dt": "điện thoại",
    "r": "rồi",
    "j": "gì",
    "nv": "nhân viên",
    "ss": "so sánh",
    "mk": "mình",
    "hd": "hoạt động",
    "sd": "sử dụng",
    "mn": "mọi người",
    "ae": "anh em",
    "v": "vậy",
    "lq": "liên quan",
    "h": "giờ",
    "lo": "lo",
    "ip": "Iphone",
    "s": "sao",
    "ms": "mới",
    "ng": "người",
    "ad": "admin",
    "cx": "cũng",
    "bt": "bình thường",
    "kh": "không",
    "a": "anh",
    "kg": "kilogram",
    "dk": "điều kiện",
    "m": "mình",
    "t": "tôi",
    "wf": "wifi",
    "hk": "hong",
    "wa": "qua",
    "la": "là",
    "vn": "Việt Nam",
    "co": "có",
    "ma": "mà",
    "ch": "chưa",
    "ns": "nói",
    "z": "vậy",
    "va": "và",
    "nt": "nhắn tin",
    "d": "dậy",
    "tr": "triệu",
    "gb": "gigabyte",
    "vv": "vân vân",
    "of": "của",
    "mh": "mấy giờ",
    "c": "cái",
    "ak": "à",
    "bh": "bây giờ",
    "x": "xem",
    "gg": "Google",
    "gi": "gì",
    "y": "ý",
    "o": "ở",
    "on": "online",
    "bn": "bạn",
    "in": "inch",
    "up": "nâng cấp",
    "da": "đã",
    "im": "im",
    "id": "ID",
    "lm": "làm",
    "bi": "bị",
    "mi": "mì",
    "tt": "tình trạng",
    "mp": "mp3",
    "tp": "thành phố",
    "pk": "phòng khách",
    "nc": "nói chuyện",
    "ah": "à",
    "ji": "gì",
    "tg": "thời gian",
    "se": "sẽ",
    "b": "bạn",
    "f": "phải",
    "ht": "hoàn thành",
    "ck": "chuyển khoản",
    "g": "giờ",
    "n": "nhé",
    "tu": "tự",
    "qa": "qua",
    "nx": "nữa",
    "sx": "sản xuất",
    "vl": "vãi",
    "de": "để",
    "tq": "trung quốc",
    "cv": "công việc",
    "ro": "rõ",
    "fm": "file manager",
    "bb": "bye bye",
    "kb": "keyboard",
    "vk": "vợ",
    "vc": "việc",
    "iu": "yêu",
    "vi": "vì",
    "vd": "ví dụ",
    "tn": "tin nhắn",
    "or": "hoặc",
    "cs": "có",
    "yt": "YouTube",
    "cu": "cứ",
    "ap": "áp",
    "th": "tháng",
    "zl": "zalo",
    "i": "y",
    "cg": "cũng",
    "km": "kilomet",
    "bk": "bếp",
    "hz": "hertz",
    "ho": "ho",
    "dx": "đấy",
    "jo": "giờ",
    "pb": "phiên bản",
    "kk": "kaka",
    "oi": "ơi",
    "pv": "phỏng vấn",
    "qc": "quảng cáo",
    "os": "operating system",
    "it": "ít",
    "le": "lẽ",
    "ni": "này",
    "w": "with",
    "sn": "sinh nhật",
    "ki": "kì",
    "lg": "lượng",
    "hg": "hông",
    "hi": "xin chào",
    "cn": "Chủ nhật",
    "sv": "sinh viên",
    "bj": "bị",
    "xg": "xong",
    "ln": "luôn",
    "xr": "xin",
    "mb": "mấy bạn",
    "za": "zalo",
    "u": "you",
    "hs": "học sinh",
    "am": "am",
    "vu": "vụ",
    "qr": "QR code",
    "re": "rẻ",
    "pt": "phát triển",
    "bo": "bỏ",
    "ly": "ly",
    "zo": "dô",
    "ny": "nay",
    "su": "sự",
    "kt": "kết thúc",
    "no": "không",
    "bx": "bổ xung",
    "vo": "vô",
    "bc": "báo cáo",
    "tl": "trả lời",
    "ja": "giá",
    "di": "đi",
    "zs": "zs",
    "ls": "lịch sử",
    "qq": "quá",
    "bg": "bây giừo",
    "ac": "anh chị",
    "ky": "ký",
    "q": "quý",
    "ty": "tình yêu",
    "ol": "online",
    "we": "chúng ta",
    "l": "lít",
    "ik": "đi",
    "tk": "tài khoản",
    "tb": "thông báo",
    "pg": "page",
    "ao": "áo",
    "hp": "hạnh phúc",
    "op": "op",
    "ud": "ud",
    "is": "là",
    "pr": "public relations",
    "wp": "wordpress",
    "pl": "làm ơn",
    "cm": "centimeter",
    "rv": "review",
    "na": "na",
    "hc": "học",
    "ha": "ha",
    "hw": "hardware",
    "nh": "nhà",
    "xi": "xì",
    "ti": "ti",
    "ze": "de",
    "hm": "hừm",
    "xm": "xem",
    "db": "database",
    "li": "li",
    "te": "te",
    "fp": "fp",
    "ju": "ju",
    "pj": "pj",
    "kv": "kv",
    "st": "status",
    "ri": "ri",
    "uc": "uc",
    "td": "td",
    "mu": "mu",
    "mo": "mo",
    "ps": "postscript",
    "ke": "kể",
    "he": "hè",
    "hy": "hy",
    "fi": "fi",
    "p": "phần",
    "my": "my",
    "rc": "rc",
    "tx": "tx",
    "dg": "đang",
    "xp": "xp",
    "mx": "mx",
    "ig": "Instagram",
    "uj": "uj",
    "ve": "về",
    "ds": "danh sách",
    "pn": "pn",
    "al": "al",
    "yd": "yd",
    "dm": "damn",
    "zi": "zi",
    "du": "du",
    "kc": "kc",
    "hf": "hf",
    "dp": "đẹp",
    "qt": "qt",
    "ph": "ph",
    "bl": "bình luận",
    "oh": "oh",
    "cb": "chuẩn bị",
    "vt": "vt",
    "dn": "dn",
    "vp": "vp"
    }
    
    # Tách văn bản thành các từ riêng lẻ
    words = text.split()
    
    # Thay thế từ viết tắt bằng dạng đầy đủ
    for i in range(len(words)):
        if words[i] in viet_tat_dict:
            words[i] = viet_tat_dict[words[i]]
    
    # Ghép các từ lại thành câu
    return ' '.join(words)


# Áp dụng hàm cho cột 'input' trong DataFrame
X_test = X_test.apply(viet_tat_to_full)

# Hiển thị DataFrame sau khi chuyển đổi
print(X_test)

0       điện thoải ổn facelock cực nhanh vân tay ôk mà...
1       mình mới mua vivo91c tải ứng_dụng games nhanh ...
2       xấu đẹp không biết ưng tgdđ phục_vụ tuyệt_vời ...
3       màn_hình hơi lác chơi game nặng mình chơi ở 2 ...
4       nói_chung máy đẹp màn amoled ổn trong tầm giáh...
                              ...                        
2219    máy chống nước ko_vậy mua ở tgdd sài ổn tầm gi...
2220    máy dùng ổn chê loa ngoài âm_thanh hơi chán mo...
2221    máy xài ngon hơn so sánh về mọi thứ màu bạc nh...
2222    dùng ổn nói_chung ngon trong tầm giá mọi người...
2223    sao mình thấy máy_ảnh vỡ kiểu nhòe vàng ai hay...
Name: comment, Length: 2224, dtype: object


In [38]:
df_train = pd.DataFrame(X_train)
df_train['label'] = y_train  

df_test = pd.DataFrame(X_test)
df_test['label'] = y_test  

# Lưu vào các tập tin CSV
df_train.to_csv("G:\\VietNamese-Sentiment-Analyst\\data\\data_processed\\train.csv", index=False)
df_test.to_csv("G:\\VietNamese-Sentiment-Analyst\\data\\data_processed\\test.csv", index=False)